# Explicação do código
Este código foi desenvolvido para processar dois conjuntos de dados: um contendo dados de pesquisas e outro com dados sobre instituições de ensino superior (IES) no Brasil. O objetivo é limpar e selecionar informações relevantes de ambos os conjuntos de dados e, em seguida, combiná-los com base no estado (UF).

Vamos dividir o código seção por seção:

Esta seção importa duas bibliotecas Python necessárias:

`pandas:` Esta é uma biblioteca fundamental para manipulação e análise de dados, fornecendo estruturas de dados como DataFrames. É comumente usada para ler, limpar, transformar e analisar dados tabulares.

`re:` Este é o módulo integrado do Python para expressões regulares, que são usadas para correspondência de padrões em strings. Neste código, ele será usado para limpar nomes de colunas.
# Parte 1: Limpeza e seleção da pesquisa "State of Datda"

In [48]:
import pandas as pd
import re

In [49]:
# --- Parte 1: Limpeza e Seleção do "State of Data" (usando sua lógica) ---

print("Iniciando Parte 1: Limpeza e seleção do 'State of Data'...")
df_survey = pd.read_csv('State_of_data_BR_2023_Kaggle - df_survey_2023.csv')

Iniciando Parte 1: Limpeza e seleção do 'State of Data'...


Esta parte começa imprimindo uma mensagem indicando o início do processo. Em seguida, carrega um arquivo CSV `'State_of_data_BR_2023_Kaggle - df_survey_2023.csv'`em um DataFrame do Pandas chamado df_survey. Este DataFrame agora contém os dados brutos da pesquisa.





In [50]:
# Sua lista de colunas fixas, com a adição da coluna de estado ('P1_i_1 ') para permitir a união
fixed_columns = [
    "('P1_a ', 'Idade')",
    "('P1_b ', 'Genero')",
    "('P1_c ', 'Cor/raca/etnia')",
    "('P1_l ', 'Nivel de Ensino')",
    "('P2_f ', 'Cargo Atual')",
    "('P2_g ', 'Nivel')",
    "('P2_h ', 'Faixa salarial')",
    "('P2_c ', 'Numero de Funcionarios')",
    "('P2_b ', 'Setor')",
    "('P2_r ', 'Atualmente qual a sua forma de trabalho?')",
    "('P2_s ', 'Qual a forma de trabalho ideal para você?')",
    "('P2_i ', 'Quanto tempo de experiência na área de dados você tem?')",
    "('P2_k ', 'Você está satisfeito na sua empresa atual?')",
    "('P1_i_1 ', 'uf onde mora')" # ADICIONADO: Essencial para a união com os dados das IES
]

Aqui, uma lista chamada` fixed_columnsé `criada. Essa lista contém os nomes de colunas específicas do conjunto de dados da pesquisa que são consideradas essenciais para a análise. Os nomes dessas colunas parecem estar em um formato que inclui um código e uma descrição. O comentário indica que a `'P1_i_1 `'coluna (que representa o estado onde a pessoa reside) é crucial para a posterior combinação desses dados com os dados do IES.

In [51]:
# Seleção das colunas dinâmicas (motivos de insatisfação e critérios de escolha)
p2_l_columns = [col for col in df_survey.columns if col.startswith("('P2_l_")]
p2_o_columns = [col for col in df_survey.columns if col.startswith("('P2_o_")]

selected_columns = fixed_columns + p2_l_columns + p2_o_columns

Esta parte identifica colunas adicionais do `df_surveyDataFrame`. Ela utiliza compreensões de lista para encontrar nomes de colunas que começam com` "('P2_l_"e "('P2_o_"`. Essas provavelmente representam motivos de insatisfação ou critérios para a escolha de uma empresa, que são consideradas colunas "dinâmicas". As colunas identificadas são então adicionadas à `fixed_columnslista` para criar uma `selected_columnslista` final.

In [52]:
# Verificação para garantir que todas as colunas existem
selected_columns = [col for col in selected_columns if col in df_survey.columns]

# Criação do DataFrame selecionado
df_selected = df_survey[selected_columns].copy()


Antes de selecionar as colunas, este código verifica se todos os nomes de colunas` selected_columnsrealmente` existem no` df_surveyDataFrame` original. Esta é uma boa prática para evitar erros caso o nome de uma coluna esteja incorreto ou ausente. Em seguida, ele cria um novo DataFrame chamado , df_selectedque contém apenas o selected_columnsfrom verificado `df_survey`. O .`copy()`é usado para garantir que quaisquer modificações` df_selectednão afetem o df_survey.`

In [53]:
# Tratamento da variável alvo
target_col = "('P2_k ', 'Você está satisfeito na sua empresa atual?')"
df_selected.loc[:, target_col] = df_selected[target_col].astype(str)
df_cleaned_survey = df_selected.dropna(subset=[target_col])
df_cleaned_survey = df_cleaned_survey[df_cleaned_survey[target_col] != 'Desconhecido']

<ipython-input-53-94c640433119>:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0.0' '1.0' '1.0' ... '1.0' 'nan' 'nan']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_selected.loc[:, target_col] = df_selected[target_col].astype(str)


Esta seção se concentra na preparação da "variável alvo", que é a coluna que indica a satisfação com a empresa atual (` '('P2_k ', 'Você está satisfeito na sua empresa atual?')'`).


* A coluna é selecionada e seu nome é armazenado em `target_col`.
* `df_selected.loc[:, target_col] = df_selected[target_col].astype(str) `converte os dados da coluna de destino para o tipo de dados string. Isso garante um tratamento consistente, independentemente do tipo de dado original.
*` df_cleaned_survey `= df_selected.dropna(subset=[target_col])remove todas as linhas df_selectedonde o valor target_colestá faltando (NaN).
*` df_cleaned_survey = df_cleaned_survey`[df_cleaned_survey[target_col] != 'Desconhecido']filtra ainda mais o DataFrame, removendo linhas cujo valor da coluna de destino é "Desconhecido". Isso resulta em um DataFrame df_cleaned_surveyque contém apenas linhas com valores de satisfação válidos e não ausentes.

In [54]:
# Preenchimento de valores ausentes conforme sua lógica
for col in df_cleaned_survey.columns:
    if col.startswith("('P2_l_") or col.startswith("('P2_o_"):
        df_cleaned_survey.loc[:, col] = df_cleaned_survey[col].fillna(0)
    else:
        df_cleaned_survey.loc[:, col] = df_cleaned_survey[col].fillna('Desconhecido')

print("'State of Data' limpo e selecionado com sucesso.")

'State of Data' limpo e selecionado com sucesso.


Este loop itera por todas as colunas no df_cleaned_surveyDataFrame.

Se o nome de uma coluna começar com "('P2_l_"ou "('P2_o_", os valores ausentes ( NaN) nessa coluna serão preenchidos com o número 0. Isso sugere que essas colunas podem representar contagens ou indicadores em que um valor ausente implica uma contagem zero.
Para todas as outras colunas, os valores ausentes são preenchidos com a string 'Desconhecido'. Esta é uma maneira de lidar com dados categóricos ausentes ou outros dados não numéricos. Por fim, uma mensagem é impressa indicando a conclusão do processo de limpeza e seleção dos dados da pesquisa.

# Etapa extra: limpeza de nomes de colunas

In [55]:
# --- Etapa Extra: Limpar nomes das colunas para a união ---

def clean_col_code(col_name):
    match = re.search(r"\'(.*?)\'", col_name)
    if match:
        return match.group(1).strip()
    return col_name

# Renomeia as colunas do dataframe já limpo
original_cleaned_columns = df_cleaned_survey.columns
df_cleaned_survey.columns = [clean_col_code(col) for col in original_cleaned_columns]

Esta seção define uma função clean_col_codeque usa expressões regulares ( re) para extrair a parte do código dos nomes das colunas (a parte entre aspas simples, por exemplo, 'P1_a '). Em seguida, ela aplica essa função a todos os nomes de colunas no df_cleaned_surveyDataFrame, substituindo os nomes originais, mais descritivos, apenas pelos códigos. Isso provavelmente é feito para simplificar os nomes das colunas e facilitar a manipulação e a mesclagem.

In [56]:
# Mapeia os códigos para nomes mais legíveis (EXPANDIDO para incluir P2_l e P2_o)
survey_rename_map = {
    'P1_a': 'Idade', 'P1_b': 'Genero', 'P1_c': 'Etnia', 'P1_l': 'Nivel_Educacao',
    'P2_f': 'Cargo_Atual', 'P2_g': 'Senioridade', 'P2_h': 'Faixa_Salarial',
    'P2_c': 'Tamanho_Empresa', 'P2_b': 'Industria', 'P2_r': 'Modelo_Trabalho_Atual',
    'P2_s': 'Modelo_Trabalho_Preferido', 'P2_i': 'Anos_Experiencia_Dados',
    'P2_k': 'Satisfacao_Empresa_Atual', 'P1_i_1': 'uf_residencia',

    # Mapeamentos para as colunas P2_l
    'P2_l_1': 'Insatisfacao_Falta_Crescimento',
    'P2_l_2': 'Insatisfacao_Salario',
    'P2_l_3': 'Insatisfacao_Relacao_Lider',
    'P2_l_4': 'Insatisfacao_Outra_Area',
    'P2_l_5': 'Insatisfacao_Beneficios',
    'P2_l_6': 'Insatisfacao_Clima_Trabalho',
    'P2_l_7': 'Insatisfacao_Falta_Maturidade_Analitica',

    # Mapeamentos para as colunas P2_o
    'P2_o_1': 'Criterio_Remuneracao',
    'P2_o_2': 'Criterio_Beneficios',
    'P2_o_3': 'Criterio_Proposito',
    'P2_o_4': 'Criterio_Flexibilidade_Remoto',
    'P2_o_5': 'Criterio_Ambiente_Clima',
    'P2_o_6': 'Criterio_Aprendizado_Referencias',
    'P2_o_7': 'Criterio_Plano_Carreira_Crescimento',
    'P2_o_8': 'Criterio_Maturidade_Tecnologia_Dados',
    'P2_o_9': 'Criterio_Qualidade_Gestores',
    'P2_o_10': 'Criterio_Reputacao_Mercado' # Renomeado para um nome mais claro
}
df_cleaned_survey = df_cleaned_survey.rename(columns=survey_rename_map)


Após limpar os nomes das colunas para apenas os códigos, esta parte cria um dicionário survey_rename_mappara mapear esses códigos para nomes mais descritivos e legíveis em inglês. Em seguida, usa o .rename()método para renomear as colunas df_cleaned_surveyde acordo com esse mapeamento.

# Parte 2: Agregação de dados do IES por estado (UF)

In [57]:
# --- Parte 2: Agregação dos Dados das IES por Estado (UF) ---

print("\nIniciando Parte 2: Agregação dos dados das IES...")
df_ies = pd.read_csv('MICRODADOS_ED_SUP_IES_2023.CSV', delimiter=';', encoding='latin1')


Iniciando Parte 2: Agregação dos dados das IES...


Esta seção começa imprimindo uma mensagem para a segunda parte do processo. Em seguida, lê outro arquivo CSV chamado 'MICRODADOS_ED_SUP_IES_2023.CSV'. Este arquivo é lido com um ponto e vírgula ( ;) como delimitador e 'latin1'como codificação, conforme especificado pelos argumentos de pd.read_csv(). Os dados são carregados em um DataFrame chamado df_ies.

In [58]:
ies_columns_map = { 'SG_UF_IES': 'uf_ies', 'QT_TEC_TOTAL': 'Qtd_Tecnicos_IES', 'QT_DOC_TOTAL': 'Qtd_Docentes_IES' }
df_ies_selected = df_ies[list(ies_columns_map.keys())].rename(columns=ies_columns_map)

Um dicionário ies_columns_mapé criado para mapear os nomes das colunas originais do conjunto de dados IES ( 'SG_UF_IES', 'QT_TEC_TOTAL', 'QT_DOC_TOTAL') para nomes mais descritivos ( 'uf_ies', 'Qtd_Tecnicos_IES', 'Qtd_Docentes_IES'). Em seguida, ele seleciona essas colunas do df_iesDataFrame e as renomeia usando o mapa fornecido, armazenando o resultado em df_ies_selected.

In [59]:
numeric_cols = ['Qtd_Tecnicos_IES', 'Qtd_Docentes_IES']
for col in numeric_cols:
    df_ies_selected[col] = pd.to_numeric(df_ies_selected[col], errors='coerce')

Este loop itera pelo valor especificado numeric_cols. Para cada coluna, ele tenta converter os dados para um tipo numérico usando pd.to_numeric(). O errors='coerce'argumento significa que, se algum valor não puder ser convertido em um número, ele será substituído por NaN(Não é um número).



In [60]:
df_ies_aggregated = df_ies_selected.groupby('uf_ies').agg({
    'Qtd_Tecnicos_IES': 'sum',
    'Qtd_Docentes_IES': 'sum',
    'uf_ies': 'count'
}).rename(columns={'uf_ies': 'Total_IES_no_Estado', 'Qtd_Tecnicos_IES': 'Total_Tecnicos_Estado', 'Qtd_Docentes_IES': 'Total_Docentes_Estado'}) # Corrected column name here
print("Dados das IES agregados com sucesso.")

Dados das IES agregados com sucesso.


Esta é uma etapa fundamental em que os dados do IES são agregados por estado ( 'uf_ies').

df_ies_selected.groupby('uf_ies')agrupa o DataFrame pelos valores na 'uf_ies'coluna.
.agg({...})então aplica funções de agregação aos dados agrupados:
Ele calcula o sumde 'Qtd_Tecnicos_IES'para cada estado.
Ele calcula o sumde 'Qtd_Docentes_IES'para cada estado.
Ele usa 'count'a 'uf_ies'coluna para contar o número de entradas IES dentro de cada estado.
.rename(...)renomeia as colunas agregadas resultantes para que sejam mais descritivas: 'uf_ies'torna-se 'Total_IES_no_Estado', 'Qtd_Tecnicos_IES'torna-se 'Total_Tecnicos_Estado'e 'Qtd_Docentes_Estado'torna-se 'Total_Docentes_Estado'. Os dados agregados são armazenados em df_ies_aggregated. Uma mensagem de sucesso é impressa.

# Parte 3: União final do banco de dados (mesclagem)

In [61]:
# --- Parte 3: União (Merge) Final das Bases de Dados ---

print("\nIniciando Parte 3: União final dos dados...")
df_final = pd.merge(
    df_cleaned_survey,
    df_ies_aggregated,
    left_on='uf_residencia',
    right_on='uf_ies',
    how='left'
)
print("União finalizada.")


Iniciando Parte 3: União final dos dados...
União finalizada.


Esta parte combina os dados limpos da pesquisa ( df_cleaned_survey) com os dados agregados do IES ( df_ies_aggregated).

pd.merge()é usado para executar a operação de junção.
df_cleaned_surveyé o DataFrame esquerdo.
df_ies_aggregatedé o DataFrame correto.
left_on='uf_residencia'especifica a coluna a df_cleaned_surveyser usada para mesclagem.
right_on='uf_ies'especifica a coluna a df_ies_aggregatedser usada para mesclagem.
how='left'indica uma mesclagem à esquerda. Isso significa que todas as linhas do df_cleaned_survey(DataFrame esquerdo) serão incluídas no resultado. Se um estado de df_cleaned_surveynão tiver uma entrada correspondente em df_ies_aggregated, as colunas de df_ies_aggregatedterão valores ausentes (NaN) para essa linha. O resultado da mesclagem é armazenado em df_final, e uma
mensagem de conclusão é impressa.

# Parte 4: Salvando o Resultado

In [62]:
# --- Parte 4: Salvando o Resultado ---
final_filename = 'dados_tratados_combinados.csv'
df_final.to_csv(final_filename, index=False, encoding='utf-8-sig')

print(f"\nProcesso concluído! Arquivo '{final_filename}' gerado com sucesso.")
print("\nVisualização das 5 primeiras linhas do arquivo final:")
print(df_final[['Idade', 'Genero', 'Faixa_Salarial', 'uf_residencia', 'Total_IES_no_Estado', 'Total_Docentes_Estado']].head())


Processo concluído! Arquivo 'dados_tratados_combinados.csv' gerado com sucesso.

Visualização das 5 primeiras linhas do arquivo final:
   Idade     Genero                    Faixa_Salarial uf_residencia  \
0     31  Masculino  de R$ 12.001/mês a R$ 16.000/mês            MG   
1     30  Masculino    de R$ 6.001/mês a R$ 8.000/mês            ES   
2     37   Feminino    de R$ 6.001/mês a R$ 8.000/mês            SP   
3     22  Masculino                      Desconhecido            SP   
4     34  Masculino    de R$ 4.001/mês a R$ 6.000/mês            MG   

   Total_IES_no_Estado  Total_Docentes_Estado  
0                310.0                41090.0  
1                 69.0                 6928.0  
2                586.0                74822.0  
3                586.0                74822.0  
4                310.0                41090.0  


Esta seção final salva o DataFrame mesclado resultante ( df_final) em um arquivo CSV.

final_filename = 'dados_finais_combinados.csv'define o nome do arquivo de saída.
df_final.to_csv(final_filename, index=False, encoding='utf-8-sig')salva o DataFrame no arquivo CSV especificado.
index=Falseimpede a gravação do índice DataFrame como uma coluna no CSV.
encoding='utf-8-sig'especifica a codificação de caracteres para o arquivo de saída. Por fim, mensagens de sucesso são impressas, incluindo o nome do arquivo gerado, e as 5 primeiras linhas de colunas específicas do df_finalDataFrame são impressas no console, permitindo uma rápida visualização dos dados combinados.